In [1]:
import sys
sys.path.append('../_amigocloud')

In [2]:
from amigocloud import AmigoCloud
import pandas as pd
from shapely import wkb

import geopandas as gpd

from datetime import datetime, timedelta
from shapely import wkb
from shapely.geometry import Polygon, MultiPolygon

In [3]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [4]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [5]:
def convertir_wkb(wkb_data):
    geom = wkb.loads(wkb_data, hex=True)
    # Si la geometría es un MultiPolygon con un solo polígono, conviértelo a Polígono
    if isinstance(geom, MultiPolygon) and len(geom.geoms) == 1:
        geom = geom.geoms[0]
    return geom

In [6]:
semana = 19

In [7]:
query = f"select cast(NULLIF(split_part(insp.semana_3ra, 'S', 2), '') as integer) as sem_reg, cast(split_part(prop.propiedad, ' / ', 1) as integer) unidad_01, split_part(prop.propiedad, ' / ', 2) unidad_02, cast(split_part(insp.canhero, ' / ', 1) as integer) unidad_03, split_part(insp.canhero, ' / ',2) unidad_04, lote.lote unidad_05, lote.superficie area, lote.fecha_siembra::date fecha_ini, 'SIEMBRA 2024' origen, lote.geometria geometry from dataset_307560 insp inner join dataset_307562 prop on insp.amigo_id = prop.insp_ref_prop inner join dataset_307563 lote on prop.amigo_id = lote.prop_ref_lote where insp.estado_3ra_insp = 'APROBADO' and (lote.estado_de_siembra = 'CAÑA NACIDA' or lote.estado_de_siembra = 'CAÑA NO NACIDA') and cast(NULLIF(split_part(insp.semana_3ra, 'S', 2), '') as integer) = {semana}"
query_consulta = {'query': query}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_consulta)
data = select['data']

In [8]:
data

[{'sem_reg': 19,
  'unidad_01': 1520,
  'unidad_02': 'VILLA COPACABANA - CRONENBOLD',
  'unidad_03': 2641,
  'unidad_04': 'CRONENBOLD AGUILERA ROBERTO',
  'unidad_05': 'L26',
  'area': 7.51,
  'fecha_ini': '2024-04-15',
  'origen': 'SIEMBRA 2024',
  'geometry': '0106000020E61000000100000001030000000100000010000000E8548DB9C6A24FC0E9D500A5A16831C0B2683177A0A24FC079E19289526831C0650AF83D8AA24FC05D04DF851E6831C0F26C7F7CCDA24FC0383CDD0F756731C05EB5DFD0ECA24FC0C2446C16A16731C03E9E77E5ECA24FC0842D01E2B06731C0010E8679E9A24FC001416D08CF6731C084DCBB2BCFA24FC0C681F5701C6831C0E183D1EECBA24FC0C38DB529216831C05A85BAE7CBA24FC0C852BDCB2E6831C0656487E4CDA24FC0533024063B6831C0F5071FF7CAA24FC0041E96B44E6831C0662AC199CAA24FC0B9EED7EB5A6831C074161587CEA24FC0AD48062E746831C0B848CB5ECFA24FC0298B3C9B816831C0E8548DB9C6A24FC0E9D500A5A16831C0'},
 {'sem_reg': 19,
  'unidad_01': 1520,
  'unidad_02': 'VILLA COPACABANA - CRONENBOLD',
  'unidad_03': 2641,
  'unidad_04': 'CRONENBOLD AGUILERA ROBERTO',
  'unidad_05': '

In [9]:
df = pd.DataFrame(data)
df['geometry'] = df['geometry'].apply(convertir_wkb)
data = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')
gdf_utm = data.to_crs('EPSG:32720')
gdf_utm['fecha_ini'] = pd.to_datetime(gdf_utm['fecha_ini'])
now = datetime.now()
gdf_utm['dias'] = (now - gdf_utm['fecha_ini']).dt.days
gdf_utm['fecha_ini'] = gdf_utm['fecha_ini'].dt.strftime('%Y-%m-%d')

In [216]:
#file_name = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2024\SHP\SIEMBRAS.shp'

# Exporta el GeoDataFrame como un shapefile
#gdf_utm.to_file(file_name)

In [10]:
file_name = r'C:\Documents\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2024\SHP\SIEMBRAS.shp'
gdf_siembras = gpd.read_file(file_name)
gdf_siembras = pd.concat([gdf_siembras, gdf_utm], ignore_index=True)
gdf_siembras.to_file(file_name)